# 性能分析
# Performance Analysis

本节我们将通过查看TopDown计数器、查看Cache MPKI数据、SPEC06算分三个部分，介绍基本的性能分析流程。

In this section, we introduce the basic performance analysis workflow through three aspects: checking TopDown counters, checking cache MPKI data, and calculating SPEC06 scores.

## 查看 TopDown 计数器
## Check TopDown counters

我们使用 gem5_data_proc 仓库下的脚本来处理上节运行 GEM5 得到的原始数据，获取 top-down 计数器。

We use the scripts in the gem5_data_proc repository to process the raw data generated by GEM5 in the previous section and obtain the TopDown counters.

In [ ]:
%%bash
cd .. && source env.sh

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s ${WORK_DIR}/04-gem5/02-first-run \
    -t --topdown-raw

## 查看 Cache MPKI数据
## Check Cache MPKI data

类似地，我们也可以查看 Cache MPKI 数据。由于上一节的 hello-world 程序缺乏代表性，这里我们改用预先运行的 SPEC06 切片结果来进行更具参考价值的分析。


Similarly, we can further examine the Cache MPKI data. As the hello-world program used in the previous section lacks representativeness, we instead use pre-executed SPEC06 slices to conduct a more insightful analysis.

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-cache-example.csv \
    --cache \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-cache-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    -o weighted.csv \
    | tail -n 33

## SPEC06 算分
## Calculate SPEC06 scores

我们还可以根据切片的权重和运行结果**估算**运行完整 SPEC06 的分数，评估整体的性能。

We can also estimate the overall performance by calculating the SPEC06 score based on the weights of the slices and their execution results.

In [ ]:
%%bash
cd .. && source env.sh

mkdir -p ${WORK_DIR}/04-gem5/03-perf
cd ${WORK_DIR}/04-gem5/03-perf

export PYTHONPATH=${GEM5_DATA_PROC_HOME}
ulimit -n 65535

python3 ${GEM5_DATA_PROC_HOME}/batch.py \
    -s $(get_asset gem5-spec-topdown-result) \
    -o gem5-score-example.csv \
    > /dev/null

mkdir -p results

python3 ${GEM5_DATA_PROC_HOME}/simpoint_cpt/compute_weighted.py \
    -r gem5-score-example.csv \
    -j $(get_asset gem5-spec-topdown-result/cluster-0-0.json) \
    --score score.csv \
    | tail -n 41